In [1]:
# 📊 FUNCIÓN LEER_HOJA ACTUALIZADA
# Funciona tanto localmente (con .env) como en GitHub Actions (con secrets)

import os
import pandas as pd
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from dotenv import load_dotenv
import json

def configurar_google_sheets():
    """
    Configura la conexión a Google Sheets
    Funciona tanto localmente (con .env) como en GitHub Actions (con secrets)
    """
    # Cargar variables de entorno
    load_dotenv()
    
    # Configuración
    SHEET_ID = os.getenv('GOOGLE_SHEET_ID')
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    
    # Método 1: Intentar usar archivo JSON local (para ejecución local)
    SERVICE_ACCOUNT_FILE = os.getenv('GOOGLE_SERVICE_ACCOUNT_FILE')
    if SERVICE_ACCOUNT_FILE and os.path.exists(SERVICE_ACCOUNT_FILE):
        print("🔑 Usando archivo JSON local para autenticación")
        creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    else:
        # Método 2: Usar JSON desde variable de entorno (para GitHub Actions)
        SERVICE_ACCOUNT_JSON = os.getenv('GOOGLE_SERVICE_ACCOUNT_JSON')
        if SERVICE_ACCOUNT_JSON:
            print("🔑 Usando JSON desde variable de entorno para autenticación")
            # Crear credenciales desde el JSON
            service_account_info = json.loads(SERVICE_ACCOUNT_JSON)
            creds = Credentials.from_service_account_info(service_account_info, scopes=SCOPES)
        else:
            raise ValueError("❌ No se encontraron credenciales de Google. Configura GOOGLE_SERVICE_ACCOUNT_FILE o GOOGLE_SERVICE_ACCOUNT_JSON")
    
    # Crear servicio
    service = build('sheets', 'v4', credentials=creds)
    
    return service, SHEET_ID

# Configurar conexión global
service, SHEET_ID = configurar_google_sheets()
print(f"✅ Conectado a Google Sheet: {SHEET_ID}")

def leer_hoja(nombre_hoja='REGISTRO_DIARIO'):
    """
    Lee TODA la hoja completa desde Google Sheets, sin importar cuántas filas tenga.
    Se adapta automáticamente al crecimiento de datos y maneja celdas vacías.
    
    Args:
        nombre_hoja (str): Nombre de la hoja a leer
        
    Returns:
        pandas.DataFrame: DataFrame con los datos de la hoja o None si hay error
    """
    try:
        # Leer toda la hoja (sin especificar rango)
        range_name = f'{nombre_hoja}'
        
        result = service.spreadsheets().values().get(
            spreadsheetId=SHEET_ID, 
            range=range_name
        ).execute()
        
        values = result.get('values', [])
        
        if values:
            if len(values) > 1:
                # Obtener encabezados y número de columnas esperadas
                headers = values[0]
                num_columns = len(headers)
                
                # Normalizar todas las filas para que tengan el mismo número de columnas
                data_rows = []
                for row in values[1:]:
                    # Si la fila tiene menos columnas, completar con cadenas vacías
                    if len(row) < num_columns:
                        row_completa = row + [''] * (num_columns - len(row))
                        data_rows.append(row_completa)
                    else:
                        data_rows.append(row)
                
                df = pd.DataFrame(data_rows, columns=headers)
            else:
                df = pd.DataFrame(values)
            
            print(f"✅ Se leyeron {len(df)} filas de {nombre_hoja}")
            print(f"📊 Columnas: {list(df.columns)}")
            return df
        else:
            print("❌ No se encontraron datos")
            return None
            
    except Exception as e:
        print(f"❌ Error al leer la hoja {nombre_hoja}: {e}")
        return None

print("🎯 Función leer_hoja() lista para usar")

🔑 Usando archivo JSON local para autenticación
✅ Conectado a Google Sheet: 14AtL1_MHWaN1JR_dbbddif9w0ujlx4wFrEuelmv6gfs
🎯 Función leer_hoja() lista para usar


In [3]:
proyectos=leer_hoja("REGISTRO_CALENDARIO")
pagosproy=leer_hoja("PAGOSPROY")
vendedores=leer_hoja("VENDEDORAS")

✅ Se leyeron 866 filas de REGISTRO_CALENDARIO
📊 Columnas: ['Colaborador', 'HoraEntrada', 'HoraSalida', 'FechaEntrada', 'FechaSalida', 'Minutos', 'Minutos_extras', 'Minutos_normales', 'ID_Calendario', 'Alerta', 'Descripcion', 'Observacion', 'Extratime', 'ID_Registro']
✅ Se leyeron 4 filas de PAGOSPROY
📊 Columnas: ['id_pago', 'DNI', 'Colaborador', 'periodo_inicio', 'periodo_fin', 'fecha_pago', 'horas_normales', 'horas_extra', 'horas_extra_no_pagadas', 'pago_base', 'monto_total', 'horas_deberia']
✅ Se leyeron 4 filas de PAGOSPROY
📊 Columnas: ['id_pago', 'DNI', 'Colaborador', 'periodo_inicio', 'periodo_fin', 'fecha_pago', 'horas_normales', 'horas_extra', 'horas_extra_no_pagadas', 'pago_base', 'monto_total', 'horas_deberia']
✅ Se leyeron 19 filas de VENDEDORAS
📊 Columnas: ['DNI', 'Colaborador', 'Contraseña', 'Correo', 'Perfil', 'Foto', 'pago_base', 'pago_por_hora', 'Moneda', 'Bono de Orden Taller', 'Tarifa Hora Extra', 'SubPerfil']
✅ Se leyeron 19 filas de VENDEDORAS
📊 Columnas: ['DNI', 'Co

In [4]:
# 🔍 ANALIZAR EL PROBLEMA DE PATRICIA SULLCA
print("🔍 Buscando registro de Patricia Sullca...")

# Filtrar registros de Patricia Sullca del 15/9/2025
patricia_registros = proyectos[
    (proyectos['Colaborador'] == 'Patricia Sullca') & 
    (proyectos['FechaEntrada'] == '15/9/2025')
]

print(f"📋 Registros encontrados para Patricia Sullca el 15/9/2025:")
print(patricia_registros.to_string(index=False))

print("\n🔍 Analizando horarios...")
print("HoraEntrada:", patricia_registros['HoraEntrada'].values)
print("HoraSalida:", patricia_registros['HoraSalida'].values)
print("Minutos_normales:", patricia_registros['Minutos_normales'].values)
print("Minutos_extras:", patricia_registros['Minutos_extras'].values)

🔍 Buscando registro de Patricia Sullca...
📋 Registros encontrados para Patricia Sullca el 15/9/2025:
    Colaborador HoraEntrada HoraSalida FechaEntrada FechaSalida Minutos Minutos_extras Minutos_normales ID_Calendario Alerta Descripcion Observacion Extratime ID_Registro
Patricia Sullca     8:01:18   13:01:50    15/9/2025   15/9/2025     300              0              300 RCL-000000859                                       No    b71f9014
Patricia Sullca    19:47:42   19:47:44    15/9/2025   15/9/2025       0            107             -107 RCL-000000865                                       No    bef20955

🔍 Analizando horarios...
HoraEntrada: ['8:01:18' '19:47:42']
HoraSalida: ['13:01:50' '19:47:44']
Minutos_normales: ['300' '-107']
Minutos_extras: ['0' '107']


In [ ]:
# 🔧 FUNCIÓN CORREGIDA PARA CALCULAR MINUTOS
from datetime import datetime

def calcular_minutos_correctamente(hora_entrada, hora_salida):
    """
    Calcula correctamente los minutos trabajados entre dos horas
    """
    try:
        # Parsear las horas
        entrada_dt = datetime.strptime(hora_entrada, "%H:%M:%S")
        salida_dt = datetime.strptime(hora_salida, "%H:%M:%S")
        
        # Calcular diferencia en segundos
        diferencia_segundos = (salida_dt - entrada_dt).total_seconds()
        
        # Si es negativo, significa que cruzó medianoche
        if diferencia_segundos < 0:
            diferencia_segundos += 24 * 60 * 60  # Agregar 24 horas
        
        # Convertir a minutos
        minutos_totales = int(diferencia_segundos / 60)
        
        return minutos_totales
        
    except ValueError as e:
        print(f"❌ Error al parsear horas {hora_entrada} - {hora_salida}: {e}")
        return 0

# 🧪 PROBAR CON EL CASO DE PATRICIA SULLCA
print("🧪 PROBANDO CALCULO CORREGIDO:")
print("Caso problematico: 19:47:42 → 19:47:44")

minutos_calculados = calcular_minutos_correctamente("19:47:42", "19:47:44")
print(f"✅ Minutos calculados correctamente: {minutos_calculados}")

# Probar con el caso normal también
print("\nCaso normal: 8:01:18 → 13:01:50")
minutos_normales = calcular_minutos_correctamente("8:01:18", "13:01:50")
print(f"✅ Minutos calculados: {minutos_normales}")

# 🔍 REVISAR TODOS LOS REGISTROS DE PATRICIA
print(f"\n📊 ANALISIS COMPLETO DE PATRICIA SULLCA:")
for index, row in patricia_registros.iterrows():
    entrada = row['HoraEntrada']
    salida = row['HoraSalida']
    minutos_sheet = row['Minutos']
    
    if entrada and salida:
        minutos_corregidos = calcular_minutos_correctamente(entrada, salida)
        print(f"   📝 {entrada} → {salida}")
        print(f"      Sheet dice: {minutos_sheet} minutos")
        print(f"      Deberia ser: {minutos_corregidos} minutos")
        
        if str(minutos_sheet) != str(minutos_corregidos):
            print(f"      🚨 DIFERENCIA DETECTADA!")
        else:
            print(f"      ✅ Correcto")
        print()